<a href="https://colab.research.google.com/github/SofiaT08/Business-Intelligence-/blob/main/Reto_2_%C3%81rboles_Decisi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Integrantes**

* Maria Jose Botero Zapata
* Manuela Velez Sosa
* Sofia Torres Roldan

0. Se procede con la carga de las librerias de trabajo

In [ ]:
#Librerias cientificas
import pandas as pd
import numpy as np

#Librerias Árbol
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#Conectar al drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Se procede con la carga de los datos

In [ ]:
nxl='/content/drive/MyDrive/AnalíticaNegocios /2. CollegeAdmisions_Int_M.xlsx'
XDB=df=pd.read_excel(nxl,sheet_name=0) #Base Collegue Admissions
XDB=XDB.dropna()
XDB.head()

,Applicant,Edu_Parent1,Edu_Parent2,Gender,White,Asian,HSGPA,SAT/ACT,College,Admitted,Enrolled,College_GPA
6,7,6,6,F,1,0,4.26,1430,Math & Science,Yes,Yes,3.956
28,29,7,6,M,1,0,3.95,1340,Math & Science,Yes,Yes,3.361
46,47,3,6,M,1,0,4.33,1200,Business & Economics,No,Yes,3.433
61,62,7,7,F,1,0,4.27,1530,Business & Economics,No,Yes,3.890
65,66,6,7,F,0,0,4.04,1270,Math & Science,Yes,Yes,3.025


2. Convertimos los datos de las variables 'Gender' y 'Admitted' a números.

In [ ]:
#Seleccionar las variables
XD=XDB[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']]
yd=XDB[['Admitted']]

XD.loc[:,'Gender']=XD['Gender'].replace({'F': 0, 'M': 1})


<ipython-input-4-57fb9301be12>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  XD.loc[:,'Gender']=XD['Gender'].replace({'F': 0, 'M': 1})


**Descripción de las variables**

La base de datos de College Admissions posee un total de 17339 datos,de los cuales se utilizan 1528 para entrenar el modelo. Cada uno de los aspirantes esta descrito bajo las siguientes 7 tipos de variables socioeconómicas consideradas variables de predicción:

* ⁠Nivel educativo de los padres: Muestra el grado de escolaridad de los padres o
tutores de los aspirantes.
* ⁠Genero: Forma en que la persona se expresan sobre su identidad género, independiente del sexo asignado al nacer.
* ⁠White: Indica si las personas son categorizadas como “blancos” en los contextos sociales.
* ⁠Asian: Indica si las personas caracterizadas como “Asiáticos” en los contextos sociales.
* ⁠High score Grade Point Average (HSGPA): Promedio de calificaciones obtenidas por el estudiante aspirante en el periodo de la secundaria.
* ⁠Scholastic Asessment Test (SAT): Examen estandarizado para medir los presaberes de los estudiantes próximos a ingresar a la universidad.

Es relevante resaltar que la variable de salida (respuesta) es "Admisión" la cual evidencia si el estudiante es admitido o no a la universidad.


4. Se procede con la implementación del modelo

In [ ]:
mar=DecisionTreeClassifier(criterion='gini',max_depth=4)
mar.fit(XD,yd)

#Se procede con la gráfica
from pydotplus import graph_from_dot_data #Me crea la gráfica
from sklearn.tree import export_graphviz #Me exporta a PNG,JPEG,TIFF

ve=['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']
#Nombres para el Árbol
dot_data=export_graphviz(mar,feature_names=ve) #Crear el gráfico
graph=graph_from_dot_data(dot_data)
graph.write_png('2.Reto2_ÁrbolDecisión.png') #Guardar la gráfica

True

5. Determinar el número de personas admitidas a la Universidad

In [ ]:
Admitted = XDB['Admitted'].value_counts().to_dict()

num_yes = Admitted.get('Yes', 0)
num_no = Admitted.get('No', 0)

print(f"Número de admitidos (Yes): {num_yes}")
print(f"Número de no admitidos (No): {num_no}")

Número de admitidos (Yes): 1303
Número de no admitidos (No): 225


6. Determinar el número de nodos puros para el árbol asociado con la estructura del modelo:

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# Asumiendo que 'mar' es tu modelo de árbol de decisión entrenado

# Obtener el número de nodos en el árbol
n_nodes = mar.tree_.node_count

# Obtener los valores de los nodos (número de muestras por clase)
values = mar.tree_.value

# Contar los nodos puros
pure_node_count = 0
for i in range(n_nodes):
    # Verificar si el nodo es una hoja (no tiene hijos)
    if mar.tree_.children_left[i] == mar.tree_.children_right[i]:
        # Verificar si el nodo es puro (uno de los valores es 0)
        if values[i][0][0] == 0 or values[i][0][1] == 0:
            pure_node_count += 1

print("Número de nodos puros:", pure_node_count)

Número de nodos puros: 7


9. Evaluar el comportamiento del modelo

In [ ]:
ydp=mar.predict(XD) #Pronóstico sobre la base de datos original
cm=confusion_matrix(yd,ydp) #Matriz de confusión
print("La matriz de confusión es:\n",cm)

VN=cm[0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1]


#Métricas de desempeño
Ex=(VP+VN)/(VP+VN+FP+FN)
print("La exactitud del modelo es:\n",Ex)
TE = (FP + FN) / (VP + VN + FP + FN)
print("La tasa de error del modelo es:\n",TE)

Se= VP/(VP+FN)
print("La sensibilidad del modelo es:\n",Se)

Sp=VN/(VN+FP)
print("La especificidad del modelo es:\n",Sp)

Pr= VP/(VP+FP)
print("La precisión del modelo es:\n",Pr)

PredNeg= VN/(VN+FN)
print("La precisión negativa del modelo es:\n",PredNeg)

La matriz de confusión es:
 [[  20  205]
 [   2 1301]]
La exactitud del modelo es:
 0.8645287958115183
La tasa de error del modelo es:
 0.1354712041884817
La sensibilidad del modelo es:
 0.9984650805832693
La especificidad del modelo es:
 0.08888888888888889
La precisión del modelo es:
 0.8638778220451527
La precisión negativa del modelo es:
 0.9090909090909091


8. Se realiza el pronóstico

In [ ]:
# Cargar el archivo Excel con dos hojas
datos = pd.read_excel ('/content/drive/MyDrive/AnalíticaNegocios /2. CollegeAdmisions_Int_M.xlsx',sheet_name=1)  # Asegúrate de que la ruta sea correcta
Pronosticar_datos = datos

XD = XDB[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']]
yd = XDB[['Admitted']]

XD.loc[:,'Gender'] = XD['Gender'].map({'F': 0, 'M': 1})


X_pronostico = Pronosticar_datos[['Edu_Parent1', 'Edu_Parent2', 'Gender', 'White', 'Asian', 'HSGPA', 'SAT/ACT']]
X_pronostico.loc[:,'Gender'] = Pronosticar_datos['Gender'].replace({'F': 0, 'M': 1})

predicciones = mar.predict(X_pronostico)
Pronosticar_datos['AdmissionPrediction'] = predicciones

# Guardar los resultados en un archivo Excel con una nueva columna de predicciones
Pronosticar_datos.to_excel('pronosticoestudiantes.xlsx', index=False)

print("Pronóstico completado y guardado en 'pronosticoestudiantes.xlsx'")

Pronóstico completado y guardado en 'pronosticoestudiantes.xlsx'


<ipython-input-9-467cfea7e9e0>:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_pronostico.loc[:,'Gender'] = Pronosticar_datos['Gender'].replace({'F': 0, 'M': 1})


**Análisis de resultados**

El modelo de árbol de decisión realizado para predecir el proceso de admisión a la universidad toma las siete variables socioeconómicas anteriormente mencionadas;sin embargo al evidenciar el diagrama del árbol se encuentra que las variables más relevantes para la toma de decisión son:HSGPA,SAT/ACT y la educación de los padres. Además, es importante mencionar que el modelo se entrenó con 1528 datos.
Por otro lado, el modelo ha demostrado un buen desempeño, pues en la matriz de confusión evidenciamos que este mostro solo dos falsos negativos, por lo que el resto de verdaderos positivos (1301) acertaron la predicción.
Al analizar las métricas, encontramos que la exactitud (0,864) y la sensibilidad(0,998) superan el 80% en promedio lo que indica que la capacidad de predicción del modelo es bastante veridica, teniendo en cuenta que esta veracidad del modelo es menos precisa para identificar a los estudiantes que no serán admitidos o los verdaderos negativos, ya que la especificidad es ligeramente menor que la sensibilidad.